In [1]:
import os
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from aerolib import *
%config IPCompleter.greedy=True

In [42]:
df = pd.read_csv("SETTINGS.txt", sep = ' ')
settings1 = dict(zip(df['NAME'], df['VALUE']))

In [43]:
df = pd.read_csv("geom.txt", sep = ' ')
geom1 = dict(zip(df['NAME'], df['VALUE']))

In [81]:
def XFoil_command_CL(settings, geom, tom, V):
    command_file=open(settings['work_dir'] + 'commands.in','w')
    command_file.write('load ' + settings['work_dir'] + settings['foil1_name'] + '.dat'+'\n'\
    + settings['foil1_name'] + '\n\
    panel\n\
    oper\n\
    visc ' + str(Re(V, ba(geom))) + '\n\
    M ' + str(settings['M']) + '\n\
    type 1\n\
    vpar\n\
    n ' + str(settings['ncr']) + '\n\
    \n\
    iter\n' + str(settings['it']) + '\n\
    pacc\n'\
    + settings['work_dir'] + 'polar.dat\n\
    \n\
    cl ' + str(CL_required(V, geom['wing_area'], tom)) +'\n\
    \n\
    \n\
    quit\n')
    command_file.close()
    return 0

def XFoil_run(settings):
    sep=os.path.sep
    run_xfoil_command = settings['XFoil_path'] + 'xfoil < ' + settings['work_dir'] + \
    'commands.in'
    os.system(run_xfoil_command)
    return 0

def XFoil_read(settings):
    aero_data_file = open(settings['work_dir'] + 'polar.dat', 'r')
    last_line = aero_data_file.readlines()[-1].split()
    aero_data_file.close()
    os.system('del ' + settings['work_dir'] + 'polar.dat')
    alpha, cl, cd = last_line[0], last_line[1], last_line[2]
    return [alpha, cl, cd]

In [84]:
def K_V_solver(geom, settings, v, AR, tom):
    polar = np.array([])
    XFoil_command_CL(settings, geom, tom, v)
    XFoil_run(settings)
    polar = np.append(polar, [[XFoil_read(settings)]])
    polar = polar.reshape(polar.size // 3, 3)
    POLAR = pd.DataFrame(polar, columns = ['alpha', 'CL', 'CD'])
    POLAR.insert(0, "V", v)
    POLAR.insert(4, "CDi", CDi(POLAR.CL, AR))
    POLAR.insert(5, "K", POLAR.CL / (POLAR.CD + POLAR.CDi))
    return polar

In [85]:
XFoil_command_CL(settings1, geom1, 1., 15.)
XFoil_run(settings1)
XFoil_read(settings1)

['-2.780', '0.0715', '0.00750']

In [88]:
p = K_V_solver(geom1, settings1, 15., 6., 0.8)
p

array([['-2.910', '0.0572', '0.00760']], dtype='<U32')

In [70]:
aero_data_file = open(settings1['work_dir'] + 'polar.dat', 'r')
lines = aero_data_file.readlines()
last_line = lines[-1].split()
aero_data_file.close()
last_line[0]

'-2.780'